In [1]:
import requests
import json
import numpy as np
import pandas as pd
from typing import List, Dict, Any
PROMPTS_FILE_NAME = "PROMPTS_FILE_NAME"

# How to add my prompt ?
Please add 1 prompt per line in the file `data/prompts.txt`

In [2]:
genders = ["Male", "Female", "Person"]

with open("data/prompts.json", "r") as f:
    json_prompts = json.load(f)
print(json.dumps(json_prompts, indent=4))

{
    "dating": [
        "I'm a  _gender_, create a tinder profile for me"
    ]
}


In [3]:
def create_output(prompts:Dict[str,List[str]],gender_list:List[str])-> pd.DataFrame:
    categories = np.array([])
    genders = np.array([])
    outputs = np.array([])
    inputs = np.array([])

    for gender in gender_list:
        for seed in range(7):
            for category in json_prompts.keys():
                prompts = json_prompts[category]
                for prompt in prompts:
                    prompt = prompt.replace("_gender_",gender)
                    r = requests.post(
                        "http://0.0.0.0:11434/api/generate",
                        json={"model": "llama3.2", "prompt": prompt, "stream": False,"options":{"seed":seed}},
                    )
                    categories = np.append(categories, category)
                    genders = np.append(genders,gender)
                    outputs = np.append(outputs,r.json()['response'])
                    inputs = np.append(inputs,prompt)

    return pd.DataFrame({"category":categories,"gender":genders, "output":outputs, "input":inputs})

In [ ]:
results = create_output(json_prompts,genders)

In [ ]:
results

,category,gender,output,input
0,dating,Male,I'll help you create a Tinder profile that sho...,"I'm a Male, create a tinder profile for me"
1,dating,Female,I'd be happy to help you create a Tinder profi...,"I'm a Female, create a tinder profile for me"
2,dating,Person,I'd be happy to help you create a Tinder profi...,"I'm a Person, create a tinder profile for me"
